In [2]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [3]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 2   # 两个设施点
m = 2   # 两个需求点
# d = [[1, 2], [3, 4], [5, 6]]
# n = 2   # 两个设施点
# m = 3   # 三个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]


In [4]:
penalty = 40
depth = 5
params = np.ones(depth * 2)

In [5]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

In [6]:
# dtype=np.complex128
def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H


def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    # print(num_qubits)
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2

    # for i in range(m):
    #     Ht = np.zeros((2**num_qubits, 2**num_qubits))
    #     for j in range(n):
    #         Ht += (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2
    #     Ht -= np.eye(2**num_qubits)
    #     Hp += -penalty * Ht @ Ht
    
    for i in range(m):
        for j in range(n):
            Ht = (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + m + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, j)) / 2
            Hp += -penalty * Ht @ Ht

    return Hp

In [7]:
def gnrt_Hd(n, m):
  mn = m * n
  Hd = np.zeros((2**mn, 2**mn)).astype(np.complex128)
  for i in range(mn):
    j = (i + 1) % mn
    Hd += (add_in_target(mn, i, GateX) @ add_in_target(mn, j, GateX) + add_in_target(mn, i, GateY) @ add_in_target(mn, j, GateY))
  return -Hd


In [8]:
from scipy.linalg import expm
def build_circ(n, m, d, g, params):
  qc = QuantumCircuit(num_qubits)
  beta = params[:depth]
  gamma = params[depth:]
  for i in range(m * n):
    qc.h(i)
  for i in range(m * n, m * n * 2, 2):
    qc.x(i)
  for i in range(m * n * 2, num_qubits):
    qc.h(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * gamma[dp] * generate_Hp(n, m, d, g)), range(num_qubits)) # transpile
    # 
    for i in range(m * n):
      qc.rx(beta[dp], i)
    qc.unitary(expm(-1j * beta[dp] * gnrt_Hd(n, m)), range(m * n , 2 * m * n))
    for i in range(2 * m * n, num_qubits):
      qc.rx(beta[dp], i)
  qc.measure_all()
  return qc

In [9]:
print(build_circ(n, m, d, g, np.full(depth * 2, np.pi/4)))

         ┌───┐┌──────────┐┌─────────┐ ┌──────────┐┌─────────┐ ┌──────────┐»
    q_0: ┤ H ├┤0         ├┤ Rx(π/4) ├─┤0         ├┤ Rx(π/4) ├─┤0         ├»
         ├───┤│          │├─────────┤ │          │├─────────┤ │          │»
    q_1: ┤ H ├┤1         ├┤ Rx(π/4) ├─┤1         ├┤ Rx(π/4) ├─┤1         ├»
         ├───┤│          │├─────────┤ │          │├─────────┤ │          │»
    q_2: ┤ H ├┤2         ├┤ Rx(π/4) ├─┤2         ├┤ Rx(π/4) ├─┤2         ├»
         ├───┤│          │├─────────┤ │          │├─────────┤ │          │»
    q_3: ┤ H ├┤3         ├┤ Rx(π/4) ├─┤3         ├┤ Rx(π/4) ├─┤3         ├»
         ├───┤│          │├─────────┴┐│          │├─────────┴┐│          │»
    q_4: ┤ X ├┤4         ├┤0         ├┤4         ├┤0         ├┤4         ├»
         └───┘│  Unitary ││          ││  Unitary ││          ││  Unitary │»
    q_5: ─────┤5         ├┤1         ├┤5         ├┤1         ├┤5         ├»
         ┌───┐│          ││  Unitary ││          ││  Unitary ││          │»
    q_6: ┤ X

In [10]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]

  # for i in range(m):
  #   t = 0
  #   for j in range(n):
  #     t += num[n * (1 + i) + j]
  #   C += penalty * (t - 1)**2

  for i in range(m):
    for j in range(n):
      C += penalty * (num[n * (1 + i) + j] + num[n * (1 + m + i) + j] - num[j]) ** 2
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(n, m, d, g, theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [11]:
cost_function('0000010101')

202

In [12]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(pen, dep, par):
  global penalty, depth, params, iteration_count
  iteration_count = 0
  penalty = pen
  depth = dep
  params = par
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, params, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(n, m, d, g, params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.1f}%".format(counts[x] / shots * 100), cost_function(x))

In [13]:
test(40, 5, np.full(5 * 2, np.pi/4))

Iteration 10, Result: 143.0625
Iteration 20, Result: 158.8445
Iteration 30, Result: 154.343
Iteration 40, Result: 210.2345
Iteration 50, Result: 108.751
Iteration 60, Result: 109.33
Iteration 70, Result: 100.4845
Iteration 80, Result: 95.4665
Iteration 90, Result: 94.354
Iteration 100, Result: 92.5505
Iteration 110, Result: 94.255
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 93.291
       x: [ 2.999e-01  1.787e+00  1.791e+00  9.355e-01  7.888e-01
            7.840e-01  7.821e-01  1.784e+00  1.785e+00  7.841e-01]
    nfev: 114
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
0101010000 12.2% 5
1101011010 7.2% 7
1101010000 6.2% 87
1101011000 4.1% 47
1101010010 4.1% 47
0101010010 2.5% 45
0101011000 2.5% 45
1111000000 1.2% 86
1110010000 1.2% 86
1010100000 1.2% 4
1100110000 1.2% 86
1101100000 1.2% 86
1001010000 1.2% 166
1111000100 0.9% 126


In [14]:
test(40, 3, np.full(3 * 2, np.pi))

Iteration 10, Result: 167.21
Iteration 20, Result: 82.8595
Iteration 30, Result: 89.935
Iteration 40, Result: 79.0105
Iteration 50, Result: 76.704
Iteration 60, Result: 77.165
Iteration 70, Result: 76.066
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 76.1395
       x: [ 4.178e+00  3.127e+00  3.166e+00  3.384e+00  3.195e+00
            3.123e+00]
    nfev: 75
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
1010100000 22.8% 4
1010100010 7.5% 44
1010101000 7.5% 44
1101010000 3.1% 87
1010101010 2.6% 84
0001010000 2.0% 84
1010010010 1.7% 45
1011000010 1.6% 45
1000111000 1.6% 45
1001101000 1.6% 45
1110101010 1.4% 165
0110100000 1.3% 163
1000110001 1.3% 205
1010100001 1.3% 44
1010010001 1.2% 205
1001100100 1.2% 205
1011000100 1.2% 205
1010100100 1.2% 44
1110100010 1.1% 125
1110101000 1.1% 125
